# tide harmonics library generator R Jupyter Notebook
Jupyter Notebook David Flater's parses latest XTide harmonic files [XTide Harmonic file](https://flaterco.com/xtide/files.html#harmonicsfiles) into [Luke Miller's](https://github.com/millerlp) useable tidal harmonics Arduino library files
by Bob Smith Jul 23, 2020
MIT LICENSE

Based on Luke Miller's Sep 9, 2012 R script https://github.com/millerlp/Tide_calculator/read_harmonicsfile.R to support his [Ardiono tide clock code](https://github.com/millerlp/Tide_calculator)

### Preparation
1. get David Flater's latest harmonic file [XTide Harmonic file](https://flaterco.com/xtide/files.html#harmonicsfiles)
2. this file is binary and must be format to text
3. use command line tool ['restore_tide_db'] found in [tcd-utils package](https://flaterco.com/files/xtide/tcd-utils-20120115.tar.bz2) distributed on the [XTide site](https://flaterco.com/xtide/files.html#extras)
4. Requires software API for reading and writing Tide Constituent Database (TCD) files [libtcd](https://flaterco.com/xtide/files.html#libtcd) with instructions [here](https://flaterco.com/xtide/libtcd.html)

### Addtional References
These functions are based on code originally written by R. Pawlowicz as part of the T_TIDE harmonic analysis 
toolbox for MATLAB original [code](http://www.eos.ubc.ca/~rich/#T_Tide in t_tide_v1.3beta.zip)

The T_TIDE toolbox is described in R. Pawlowicz, B. Beardsley, and S. Lentz's "Classical tidal harmonic analysis including error estimates in MATLAB using T_TIDE", Computers and Geosciences 28 (2002), 929-937.

In [111]:
stationID = 'Indian River, Delaware'    # search string for station 8558690 38.6100° N, 75.0700° W
GMToffset = 5                           # must be Local Standard Time (LST) not Local Daylight Time (LDT)

load('Harmonics_20181227.Rdata')
stInd = grep(stationID, harms$station)
if (length(stInd) < 1) cat('\a No matching site found\n')

libname = gsub('[ ]', '', stationID)
libname = gsub('[,()&.\\]', '', libname)

if (exists("newstInd")) {
	statname = harms$station[stInd[newstInd]]
	comloc = regexpr(',', statname)
	statname = substr(statname, 1, comloc-1)
	statname = gsub('[ ]','', statname)
	libname = paste(libname, statname, sep = '')
}

In [112]:
# Create tides.cpp file

f <- file('tides.cpp', open = "wt")
cat('// tide library for station name ')
cat(libname)
cat('\n')
cat("#include \"tides.h\"\n\n")
cat('unsigned int YearIndx = 0;\n')
cat('float currHours = 0;\n')
cat('const int adjustGMT = ')
cat(GMToffset)
cat(';')
cat('char stationID[] = "', harms1$station, '";\n', sep = '')
cat('const long stationIDnumber = ', harms1$stationIDnumber, ';\n', sep = '')
cat('const float Datum =', harms1$datum, ';\n')
cat(harms1$name, sep = ', ')
cat('\n')
cat("typedef float PROGMEM prog_float_t;\n")
cat('const prog_float_t Amp[] PROGMEM = {')
cat(harms1$A, sep = ',')
cat('};\n')
cat('const prog_float_t Kappa[] PROGMEM = {')
cat(harms1$kappa, sep = ',')
cat('};\n')
cat('const prog_float_t Speed[] PROGMEM = {')
cat(harms1$speed, sep = ',')
cat('};\n')
cat('const prog_float_t Equilarg[')
cat(keep.years)
cat('][37] PROGMEM = { \n')
for (i in 1:(ncol(harms1$equilarg)-1)) {
	cat('{')
	cat(harms1$equilarg[, i], sep = ',')
	cat('},\n')
}
cat('{')
cat(harms1$equilarg[ ,ncol(harms1$equilarg)], sep = ',')
cat('} \n };\n')
cat('\n')
cat('const prog_float_t Nodefactor[')
cat(keep.years)
cat('][37] PROGMEM = { \n')
for (i in 1:(ncol(harms1$nodefactor)-1)) {
	cat('{')
	cat(harms1$nodefactor[, i], sep = ',')
	cat('},\n')
}
cat('{')
cat(harms1$nodefactor[ ,ncol(harms1$nodefactor)], sep = ',')
cat('}\n')
cat('};\n')
cat('\n')
cat(yr.start, sep = '       ')
cat('\n')
cat('const unsigned long startSecs[] PROGMEM = {')
cat(yr.unix, sep = ',')
cat('};\n')
cat('\n')
cat('const unsigned int startYear = ')
cat(yr.start[1], ';\n', sep = '')
cat('float currAmp, currSpeed, currNodefactor, currEquilarg, currKappa, tideHeight;\n\n')
cat('TideCalc::TideCalc(void){}\n\n')
cat('char* TideCalc::returnStationID(void){\n')
cat('	return stationID;\n')
cat('}\n')
cat('\n')
cat('long TideCalc::returnStationIDnumber(void){\n')
cat('	return stationIDnumber;\n')
cat('}\n')
cat('\n')
cat('float TideCalc::currentTide(DateTime now) {\n')
cat('	YearIndx = now.year() - startYear;\n ')
cat('	currHours = (now.unixtime() - pgm_read_dword_near(&startSecs[YearIndx])) / float(3600);\n')
cat('	currHours = currHours + adjustGMT;\n')
cat('	tideHeight = Datum; // initialize results variable, units of feet.\n')
cat('	for (int harms = 0; harms < 37; harms++) {\n')
cat('		currNodefactor = pgm_read_float_near(&Nodefactor[YearIndx][harms]);\n')
cat('		currAmp = pgm_read_float_near(&Amp[harms]);\n')
cat('		currEquilarg = pgm_read_float_near(&Equilarg[YearIndx][harms]);\n')
cat('		currKappa = pgm_read_float_near(&Kappa[harms]);\n')
cat('		currSpeed = pgm_read_float_near(&Speed[harms]);\n')
cat('		tideHeight = tideHeight + (currNodefactor * currAmp *\n')
cat('		 	cos( (currSpeed * currHours + currEquilarg - currKappa) * DEG_TO_RAD));\n')
cat('	}\n')
cat('return tideHeight;\n')
cat('}\n')
close(f)

In [113]:
# Create tides.h file

f <- file('tides.h', open = "wt")
cat('// tide library for station name ')
cat(libname)
cat('\n')
cat("#include \"tides.h\"\n\n")
cat("#ifndef tides_h\n")
cat("#define tides_h\n")
cat('class TideCalc {\n')
cat(' public:\n')
cat('	TideCalc();\n')
cat('		float currentTide(DateTime now);\n')
cat('		char* returnStationID(void);\n')
cat('		long returnStationIDnumber(void);\n')
cat('};\n')
cat("#endif")
close(f)

In [114]:
# Create keywords.txt linting file

f <- file('keywords.txt', open = "wt")
cat <- function(...) base::cat(..., file = f)
cat('TideCalc\tKEYWORD1\n')
cat('currentTide\tKEYWORD2\n')
cat('returnStationID\tKEYWORD2\n')
close(f)